In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool,CatBoost, CatBoostRegressor
from collections import Counter

In [2]:
cust_embs=np.load('cust_embs.npy', allow_pickle=True).item()

In [3]:
def metric(predicts, Ys):
    counter = Counter(Ys)
    max_value = counter['view']*0.1+counter['skip']*0.1+counter['like']*0.5+counter['dislike']*10
    value = 0
    for y,p in zip(Ys,predicts):
        if y == "dislike":
            value-=10*p
        if y == "like":
            value+=0.5*p
        if y =='view':
            value+=0.1*p
        if y =='skip':
            value-=0.1*p
    return value/max_value
            

In [4]:
data_path = 'dataset/'
customer_train = pd.read_csv(f'{data_path}customer_train.csv')
customer_test = pd.read_csv(f'{data_path}customer_test.csv')
stories_description = pd.read_csv(f'{data_path}stories_description.csv')
stories_reaction_test = pd.read_csv(f'{data_path}stories_reaction_test.csv')
stories_reaction_test['event_dttm']=pd.to_datetime(stories_reaction_test['event_dttm'],infer_datetime_format=True)
stories_reaction_train = pd.read_csv(f'{data_path}stories_reaction_train.csv')
stories_reaction_train['event_dttm']=pd.to_datetime(stories_reaction_train['event_dttm'],infer_datetime_format=True)
transactions = pd.read_csv(f'{data_path}transactions.csv')


sample_submit = pd.read_csv(f'{data_path}sample_submit.csv')

In [5]:
train_and_val = stories_reaction_train.merge(customer_train, on='customer_id', how='left')
test = stories_reaction_test.merge(customer_test, on='customer_id', how='left')

train_and_val['gender_cd']=train_and_val.gender_cd.map(lambda x: 1 if x =="M" else(0 if x =="F" else -1))
test['gender_cd']=test.gender_cd.map(lambda x: 1 if x =="M" else(0 if x =="F" else -1))
test['children_cnt']=test.children_cnt
train_and_val['children_cnt']=train_and_val.children_cnt

In [6]:
train_dates = stories_reaction_train['event_dttm'].astype(np.int64)
test_dates = stories_reaction_test['event_dttm'].astype(np.int64)
period = train_dates.max() - (test_dates.max() - test_dates.min())

In [7]:
val = train_and_val[train_and_val.event_dttm.astype(np.int64) > period]
train = train_and_val[train_and_val.event_dttm.astype(np.int64) <= period]

In [8]:
train=train.sort_values('event_dttm')
val=val.sort_values('event_dttm')

In [9]:
524287 in train.customer_id.values

True

In [10]:
train['cust_embs']=train.customer_id.map(lambda x:cust_embs[x][0] )
val['cust_embs']=val.customer_id.map(lambda x:cust_embs[x][0] )

In [11]:
len(train), len(val) 

(320248, 152893)

In [12]:
train=train.fillna(-999)
val=val.fillna(-999)
test=test.fillna(-999)

In [13]:
def reaction_to_weight(reaction):
    if reaction =='dislike':
        return 10.0
#         return 10.0
    if reaction =='like':
        return 0.5
#         return 0.5
    if reaction =='view':
        return 0.1
#         return 0.1
    if reaction =='skip':
        return 0.1
#         return 0.1
    
def reaction_to_class(reaction):
    if reaction =='dislike':
        return -1
    if reaction =='like':
        return 1
    if reaction =='view':
        return 1
    if reaction =='skip':
        return -1
    

In [14]:
train_class = train.event.values
trainY = [reaction_to_class(reaction)  for reaction in train.event.values]
train_weights = [reaction_to_weight(reaction) for reaction in train.event.values]
# del train['event']
val_class = val.event.values
valY = [reaction_to_class(reaction) for reaction in val.event.values]
val_weights = [reaction_to_weight(reaction) for reaction in val.event.values]
# del val['event']



In [15]:
columns = [
    'cust_embs',
                         'gender_cd',
     'job_title',
    'marital_status_cd',
    'job_position_cd',
                         'children_cnt',
                         'story_id',
                         'product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6'
                        ]
cat_columns = [
     'job_title',
    'marital_status_cd',
    'job_position_cd',
               'story_id',
               'product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6',
              ]

val_data = Pool(val[columns],
                valY,
                 cat_features=cat_columns,
               weight=val_weights
               )
train_data = Pool(train[columns],
                  trainY,
                  cat_features=cat_columns,
                 weight=train_weights
                 )

In [16]:
model = CatBoostRegressor(iterations=4000,
                           depth=2, 
#                            scale_pos_weight=1/pds[0].isLiked.mean(),
                           learning_rate=0.05,
                           loss_function='MAE', 
#                           has_time=True,
#                            logging_level='Verbose',
#                            eval_metric='AUC',
#                            bagging_temperature=1.8,
#                            max_bin=128,
#                            random_strength=0,
                           max_ctr_complexity=0,
#                            l2_leaf_reg=10,
# #                            rsm=0.2
#                            use_best_model=False,
                           task_type='GPU',
# #                            used_ram_limit='11GB'
                 
                          )

print('model')
#train the model
model.fit(train_data,
          eval_set=val_data
         )

# 999:	learn: 0.7219157	test: 0.7182027	best: 0.7142853 (747)	total: 5.63s	remaining: 0us
# only customer_id
# train metric 0.3400194482676537
# val metric 0.2889284055655854

# 999:	learn: 0.7134417	test: 0.7149829	best: 0.7147067 (529)	total: 6.47s	remaining: 0us
#  customer_id gender_cd
# train metric 0.3463319497460446
# val metric 0.2875417257135316

# 1999:	learn: 0.7161376	test: 0.7143624	best: 0.7131135 (912)	total: 12.1s	remaining: 0us
#  customer_id gender_cd children_cnt
# train metric 0.3508508692598425
# val metric 0.28944819862286486

model
0:	learn: 0.9936613	test: 0.9962126	best: 0.9962126 (0)	total: 4.54ms	remaining: 18.2s
1:	learn: 0.9874033	test: 0.9922136	best: 0.9922136 (1)	total: 8.82ms	remaining: 17.6s
2:	learn: 0.9811463	test: 0.9882153	best: 0.9882153 (2)	total: 15.2ms	remaining: 20.3s
3:	learn: 0.9748398	test: 0.9845426	best: 0.9845426 (3)	total: 18.3ms	remaining: 18.3s
4:	learn: 0.9685408	test: 0.9808369	best: 0.9808369 (4)	total: 20.8ms	remaining: 16.6s
5:	learn: 0.9622899	test: 0.9768494	best: 0.9768494 (5)	total: 26.2ms	remaining: 17.5s
6:	learn: 0.9560365	test: 0.9731892	best: 0.9731892 (6)	total: 30.6ms	remaining: 17.5s
7:	learn: 0.9497365	test: 0.9694918	best: 0.9694918 (7)	total: 34.3ms	remaining: 17.1s
8:	learn: 0.9434806	test: 0.9654876	best: 0.9654876 (8)	total: 36.5ms	remaining: 16.2s
9:	learn: 0.9371750	test: 0.9618156	best: 0.9618156 (9)	total: 40.2ms	remaining: 16s
10:	learn: 0.9309233	test: 0.9578288	best: 0.9578288 (10)	total: 45.6ms	remaining: 16.5s
11:	learn: 0.9246668	test: 0.9538290	

146:	learn: 0.5454831	test: 0.7302921	best: 0.7272022 (120)	total: 393ms	remaining: 10.3s
147:	learn: 0.5447013	test: 0.7310196	best: 0.7272022 (120)	total: 396ms	remaining: 10.3s
148:	learn: 0.5471378	test: 0.7318638	best: 0.7272022 (120)	total: 399ms	remaining: 10.3s
149:	learn: 0.5430357	test: 0.7284505	best: 0.7272022 (120)	total: 401ms	remaining: 10.3s
150:	learn: 0.5463593	test: 0.7303185	best: 0.7272022 (120)	total: 404ms	remaining: 10.3s
151:	learn: 0.5429311	test: 0.7288491	best: 0.7272022 (120)	total: 406ms	remaining: 10.3s
152:	learn: 0.5462361	test: 0.7321119	best: 0.7272022 (120)	total: 409ms	remaining: 10.3s
153:	learn: 0.5452945	test: 0.7307820	best: 0.7272022 (120)	total: 412ms	remaining: 10.3s
154:	learn: 0.5463822	test: 0.7321573	best: 0.7272022 (120)	total: 414ms	remaining: 10.3s
155:	learn: 0.5433584	test: 0.7294294	best: 0.7272022 (120)	total: 417ms	remaining: 10.3s
156:	learn: 0.5438150	test: 0.7319342	best: 0.7272022 (120)	total: 420ms	remaining: 10.3s
157:	learn

291:	learn: 0.5453433	test: 0.7333362	best: 0.7272022 (120)	total: 780ms	remaining: 9.9s
292:	learn: 0.5410961	test: 0.7299854	best: 0.7272022 (120)	total: 784ms	remaining: 9.92s
293:	learn: 0.5446292	test: 0.7333883	best: 0.7272022 (120)	total: 786ms	remaining: 9.91s
294:	learn: 0.5419039	test: 0.7306293	best: 0.7272022 (120)	total: 789ms	remaining: 9.91s
295:	learn: 0.5415502	test: 0.7306092	best: 0.7272022 (120)	total: 791ms	remaining: 9.9s
296:	learn: 0.5453654	test: 0.7330025	best: 0.7272022 (120)	total: 797ms	remaining: 9.94s
297:	learn: 0.5406929	test: 0.7301354	best: 0.7272022 (120)	total: 799ms	remaining: 9.93s
298:	learn: 0.5443636	test: 0.7335580	best: 0.7272022 (120)	total: 801ms	remaining: 9.92s
299:	learn: 0.5431676	test: 0.7315337	best: 0.7272022 (120)	total: 804ms	remaining: 9.91s
300:	learn: 0.5403295	test: 0.7303755	best: 0.7272022 (120)	total: 805ms	remaining: 9.9s
301:	learn: 0.5448232	test: 0.7331246	best: 0.7272022 (120)	total: 808ms	remaining: 9.89s
302:	learn: 0

458:	learn: 0.5453283	test: 0.7326637	best: 0.7272022 (120)	total: 1.17s	remaining: 9.01s
459:	learn: 0.5405632	test: 0.7291385	best: 0.7272022 (120)	total: 1.17s	remaining: 9.01s
460:	learn: 0.5439852	test: 0.7326682	best: 0.7272022 (120)	total: 1.17s	remaining: 9.01s
461:	learn: 0.5437951	test: 0.7308638	best: 0.7272022 (120)	total: 1.18s	remaining: 9.01s
462:	learn: 0.5404734	test: 0.7289727	best: 0.7272022 (120)	total: 1.18s	remaining: 9.01s
463:	learn: 0.5452554	test: 0.7308594	best: 0.7272022 (120)	total: 1.18s	remaining: 9s
464:	learn: 0.5414457	test: 0.7293468	best: 0.7272022 (120)	total: 1.18s	remaining: 9s
465:	learn: 0.5431096	test: 0.7308565	best: 0.7272022 (120)	total: 1.19s	remaining: 9s
466:	learn: 0.5421612	test: 0.7305845	best: 0.7272022 (120)	total: 1.19s	remaining: 9s
467:	learn: 0.5434333	test: 0.7306783	best: 0.7272022 (120)	total: 1.19s	remaining: 8.99s
468:	learn: 0.5429263	test: 0.7299275	best: 0.7272022 (120)	total: 1.2s	remaining: 9s
469:	learn: 0.5399168	test

630:	learn: 0.5436975	test: 0.7312217	best: 0.7272022 (120)	total: 1.55s	remaining: 8.31s
631:	learn: 0.5430830	test: 0.7305281	best: 0.7272022 (120)	total: 1.56s	remaining: 8.3s
632:	learn: 0.5416914	test: 0.7293254	best: 0.7272022 (120)	total: 1.56s	remaining: 8.3s
633:	learn: 0.5435241	test: 0.7302397	best: 0.7272022 (120)	total: 1.56s	remaining: 8.3s
634:	learn: 0.5399767	test: 0.7284800	best: 0.7272022 (120)	total: 1.57s	remaining: 8.3s
635:	learn: 0.5457112	test: 0.7329844	best: 0.7272022 (120)	total: 1.57s	remaining: 8.3s
636:	learn: 0.5411583	test: 0.7281819	best: 0.7272022 (120)	total: 1.57s	remaining: 8.29s
637:	learn: 0.5420522	test: 0.7302821	best: 0.7272022 (120)	total: 1.57s	remaining: 8.29s
638:	learn: 0.5441682	test: 0.7310767	best: 0.7272022 (120)	total: 1.57s	remaining: 8.29s
639:	learn: 0.5393790	test: 0.7271675	best: 0.7271675 (639)	total: 1.58s	remaining: 8.28s
640:	learn: 0.5450987	test: 0.7314233	best: 0.7271675 (639)	total: 1.58s	remaining: 8.28s
641:	learn: 0.5

...

3359:	learn: 0.5438707	test: 0.7285338	best: 0.7240337 (2116)	total: 7.76s	remaining: 1.48s
3360:	learn: 0.5399868	test: 0.7257534	best: 0.7240337 (2116)	total: 7.76s	remaining: 1.48s
3361:	learn: 0.5415105	test: 0.7278407	best: 0.7240337 (2116)	total: 7.76s	remaining: 1.47s
3362:	learn: 0.5423330	test: 0.7266367	best: 0.7240337 (2116)	total: 7.77s	remaining: 1.47s
3363:	learn: 0.5398819	test: 0.7265078	best: 0.7240337 (2116)	total: 7.77s	remaining: 1.47s
3364:	learn: 0.5432004	test: 0.7281524	best: 0.7240337 (2116)	total: 7.77s	remaining: 1.47s
3365:	learn: 0.5396994	test: 0.7251306	best: 0.7240337 (2116)	total: 7.77s	remaining: 1.46s
3366:	learn: 0.5428277	test: 0.7281139	best: 0.7240337 (2116)	total: 7.78s	remaining: 1.46s
3367:	learn: 0.5408440	test: 0.7267604	best: 0.7240337 (2116)	total: 7.78s	remaining: 1.46s
3368:	learn: 0.5417310	test: 0.7267205	best: 0.7240337 (2116)	total: 7.78s	remaining: 1.46s
3369:	learn: 0.5414194	test: 0.7261817	best: 0.7240337 (2116)	total: 7.78s	remai

3515:	learn: 0.5406289	test: 0.7261731	best: 0.7240337 (2116)	total: 8.15s	remaining: 1.12s
3516:	learn: 0.5427102	test: 0.7268919	best: 0.7240337 (2116)	total: 8.16s	remaining: 1.12s
3517:	learn: 0.5389374	test: 0.7255983	best: 0.7240337 (2116)	total: 8.16s	remaining: 1.12s
3518:	learn: 0.5437371	test: 0.7284257	best: 0.7240337 (2116)	total: 8.16s	remaining: 1.11s
3519:	learn: 0.5400237	test: 0.7261141	best: 0.7240337 (2116)	total: 8.17s	remaining: 1.11s
3520:	learn: 0.5418199	test: 0.7280739	best: 0.7240337 (2116)	total: 8.17s	remaining: 1.11s
3521:	learn: 0.5434213	test: 0.7277542	best: 0.7240337 (2116)	total: 8.17s	remaining: 1.11s
3522:	learn: 0.5399549	test: 0.7261046	best: 0.7240337 (2116)	total: 8.18s	remaining: 1.11s
3523:	learn: 0.5432152	test: 0.7276902	best: 0.7240337 (2116)	total: 8.18s	remaining: 1.1s
3524:	learn: 0.5403067	test: 0.7260949	best: 0.7240337 (2116)	total: 8.18s	remaining: 1.1s
3525:	learn: 0.5435136	test: 0.7290543	best: 0.7240337 (2116)	total: 8.19s	remaini

3637:	learn: 0.5448564	test: 0.7286547	best: 0.7240337 (2116)	total: 8.54s	remaining: 850ms
3638:	learn: 0.5390342	test: 0.7253656	best: 0.7240337 (2116)	total: 8.54s	remaining: 847ms
3639:	learn: 0.5416361	test: 0.7280017	best: 0.7240337 (2116)	total: 8.54s	remaining: 845ms
3640:	learn: 0.5429032	test: 0.7271070	best: 0.7240337 (2116)	total: 8.55s	remaining: 843ms
3641:	learn: 0.5388388	test: 0.7256628	best: 0.7240337 (2116)	total: 8.55s	remaining: 840ms
3642:	learn: 0.5441278	test: 0.7289295	best: 0.7240337 (2116)	total: 8.55s	remaining: 838ms
3643:	learn: 0.5404183	test: 0.7261204	best: 0.7240337 (2116)	total: 8.55s	remaining: 836ms
3644:	learn: 0.5415693	test: 0.7283289	best: 0.7240337 (2116)	total: 8.56s	remaining: 833ms
3645:	learn: 0.5422506	test: 0.7282885	best: 0.7240337 (2116)	total: 8.56s	remaining: 831ms
3646:	learn: 0.5402067	test: 0.7266034	best: 0.7240337 (2116)	total: 8.56s	remaining: 829ms
3647:	learn: 0.5441978	test: 0.7288520	best: 0.7240337 (2116)	total: 8.57s	remai

3771:	learn: 0.5424523	test: 0.7275558	best: 0.7240337 (2116)	total: 8.93s	remaining: 540ms
3772:	learn: 0.5404702	test: 0.7266586	best: 0.7240337 (2116)	total: 8.93s	remaining: 537ms
3773:	learn: 0.5431134	test: 0.7277088	best: 0.7240337 (2116)	total: 8.93s	remaining: 535ms
3774:	learn: 0.5401892	test: 0.7269144	best: 0.7240337 (2116)	total: 8.94s	remaining: 533ms
3775:	learn: 0.5432453	test: 0.7286876	best: 0.7240337 (2116)	total: 8.94s	remaining: 530ms
3776:	learn: 0.5417630	test: 0.7269878	best: 0.7240337 (2116)	total: 8.94s	remaining: 528ms
3777:	learn: 0.5395920	test: 0.7262682	best: 0.7240337 (2116)	total: 8.94s	remaining: 526ms
3778:	learn: 0.5437765	test: 0.7276449	best: 0.7240337 (2116)	total: 8.95s	remaining: 523ms
3779:	learn: 0.5400906	test: 0.7265472	best: 0.7240337 (2116)	total: 8.95s	remaining: 521ms
3780:	learn: 0.5421446	test: 0.7281846	best: 0.7240337 (2116)	total: 8.95s	remaining: 519ms
3781:	learn: 0.5418574	test: 0.7273093	best: 0.7240337 (2116)	total: 8.95s	remai

3941:	learn: 0.5445327	test: 0.7285312	best: 0.7240337 (2116)	total: 9.32s	remaining: 137ms
3942:	learn: 0.5396839	test: 0.7246096	best: 0.7240337 (2116)	total: 9.32s	remaining: 135ms
3943:	learn: 0.5418394	test: 0.7273490	best: 0.7240337 (2116)	total: 9.32s	remaining: 132ms
3944:	learn: 0.5422457	test: 0.7259896	best: 0.7240337 (2116)	total: 9.32s	remaining: 130ms
3945:	learn: 0.5387770	test: 0.7248646	best: 0.7240337 (2116)	total: 9.33s	remaining: 128ms
3946:	learn: 0.5445568	test: 0.7274591	best: 0.7240337 (2116)	total: 9.33s	remaining: 125ms
3947:	learn: 0.5392794	test: 0.7244376	best: 0.7240337 (2116)	total: 9.33s	remaining: 123ms
3948:	learn: 0.5425598	test: 0.7277877	best: 0.7240337 (2116)	total: 9.33s	remaining: 121ms
3949:	learn: 0.5418801	test: 0.7265823	best: 0.7240337 (2116)	total: 9.34s	remaining: 118ms
3950:	learn: 0.5397100	test: 0.7249939	best: 0.7240337 (2116)	total: 9.34s	remaining: 116ms
3951:	learn: 0.5428056	test: 0.7268641	best: 0.7240337 (2116)	total: 9.34s	remai

In [17]:
x = model.get_feature_importance(train_data)
for i in range(len(x)):
    print(columns[i], x[i])

cust_embs 100.0
gender_cd 0.0
job_title 0.0
marital_status_cd 0.0
job_position_cd 0.0
children_cnt 0.0
story_id 0.0
product_0 0.0
product_1 0.0
product_2 0.0
product_3 0.0
product_4 0.0
product_5 0.0
product_6 0.0


In [18]:
train_predict = model.predict(train_data)
val_predict = model.predict(val_data)

print('train metric ' + str(metric(train_predict.clip(-1,1), train_class)))
print('val metric ' + str(metric(val_predict.clip(-1,1), val_class)))

def mae2(predicts, Y, weights):
#     return np.mean(np.multiply(np.abs(Y-predicts), weights))
    max_value = sum(np.abs(np.multiply(Y, weights)))

    sign_weights = np.multiply(Y, weights)
    return np.sum(np.abs(sign_weights-np.multiply(predicts, weights)))/max_value


#     return np.mean(np.multiply(Y,weights)-np.multiply(predicts,weights))
# print(mae2(val_predict ,valY, val_weights))
print(mae2(train_predict ,trainY, train_weights))

train metric 0.46171661401240094
val metric 0.276987189494091
0.5386210683186494


In [22]:
val['predict']=model.predict(val_data)
val.loc[~val.customer_id.isin(set(train.customer_id.values)),'predict'] =-1
print('val metric ' + str(metric(val['predict'].clip(-1,1), val_class)))


val metric 0.28495626177725697


In [100]:
train_weights

[0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.5,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 10.0,
 0.1,
 0.1,
 0.1,
 10.0,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 10.0,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 

In [101]:
def mae2(predicts, Y, weights):
    max_value = sum(np.abs(weights))
    sign_weights = np.multiply(Y, weights)
    return np.sum(np.multiply(sign_weights, -predicts))/max_value
print(mae2(val_predict ,valY, val_weights))

-0.28253761622416307


In [91]:
np.multiply(sign_weights, -predicts)
np.abs(sign_weights-np.multiply(predicts, weights))

NameError: name 'Y' is not defined

In [ ]:
np.multiply(np.multiply(Y,weights), predicts)

0.22792780076833008


0.013162910807036882

In [102]:
mae2(valY, val_weights, val_predict)

0.0747334197951413

In [103]:
mae2(train_predict, trainY, train_weights)

0.2895163131422259

In [65]:
test_id = test.answer_id
del test['answer_id']

In [73]:
test_id

0              0
1              1
2              2
3              3
4              4
           ...  
172044    172044
172045    172045
172046    172046
172047    172047
172048    172048
Name: answer_id, Length: 172049, dtype: int64

In [66]:
test['score'] = (model.predict_proba(test)[:,1]-0.5)*2
test['answer_id'] = test_id

AttributeError: 'CatBoostRegressor' object has no attribute 'predict_proba'

In [119]:
test[['answer_id', 'score']].to_csv("predict.csv", index=False, float_format='%.3f')